# Extracting Structured JSON using Claude and Tool Use

In this cookbook, we'll explore various examples of using Claude and the tool use feature to extract structured JSON data from different types of input. We'll define custom tools that prompt Claude to generate well-structured JSON output for tasks such as summarization, entity extraction, sentiment analysis, and more.

If you want to get structured JSON data without using tools, take a look at our "[How to enable JSON mode](https://github.com/anthropics/anthropic-cookbook/blob/main/misc/how_to_enable_json_mode.ipynb)" cookbook.

## Set up the environment

First, let's install the required libraries and set up the Anthropic API client.

In [2]:
import boto3
from pydantic import BaseModel, EmailStr, Field
from typing import Optional
import json, requests

client = boto3.client("bedrock-runtime", region_name="us-west-2")
MODEL_NAME = "anthropic.claude-3-sonnet-20240229-v1:0"


## Example: Information extraction
In this example, we'll use Claude to extract information from an email thread and return the extracted information in a JSON format

In [3]:
tools = [
    {
        "toolSpec": {
            "name": "extract_email",
            "description": "Extract the following details from the emails provided",
            "inputSchema": {
                "json": {
                    "type": "object",
                    "properties": {
                        "entities": {
                            "type": "array",
                            "items": {
                                "type": "object",
                                "properties": {
                                    "SenderName": {"type": "string", "description": "The name of the sender of the parcel"},
                                    "SenderAddress": {"type": "string", "description": "The Address of the sender of the parcel"},
                                    "ReceiverName": {"type": "string", "description": "The name of the parcel receiver"},
                                    "ReceiverAddress": {"type": "string", "description": "The address of the parcel receiver"},
                                    "NumberOfParcels": {"type": "string", "description": "The number of parcels being sent"},
                                    "WeightPerParcels": {"type": "string", "description": "List containing individual weights separated by comma including the unit used"},
                                    "TotalWeightOfParcels": {"type": "string", "description": "Total weight of parcels being send, including the unit"},
                                    "Price": {"type": "number", "description": "The price to send the parcels"},
                                    "PriceCurrency": {"type": "string", "description": "The price currency"},
                                    "DeliveryTimeframe": {"type": "string", "description": "The delivery timeframe for the parcels"}
                                },
                                "required": [
                                    "SenderName", 
                                    "SenderAddress", 
                                    "ReceiverName", 
                                    "ReceiverAddress", 
                                    "NumberOfParcels", 
                                    "WeightPerParcels", 
                                    "TotalWeightOfParcels", 
                                    "Price", 
                                    "PriceCurrency",
                                    "DeliveryTimeframe"
                                ]
                            }
                        }
                    },
                    "required": ["entities"]
                }
            }
        }
    }
]

emails = """
Subject: Request shipping from Austin to New York City
 
Dear Sir or Madam,

I am looking to ship 2 parcels from Austin to New York City and would like to request a shipping quote. 

The details are:

Sender: 
John Chapman
2418 Spring Ln #5362
Austin, TX 78703
United States

Recipient:  
Anna Chapman
240 E 27th St
New York, NY 10016 
United States

Package Details:
- 2 parcels 
- Each parcel weighs 500 grams

Please let me know the estimated cost and timeline for shipping these 2 parcels from Austin to the New York City address above. 

Thank you in advance for providing a shipping quote. Please let me know if you need any additional details from me. I look forward to hearing back from you.

Best,
John Chapman



Subject: RE: Request shipping from Austin to New York City


Dear Mr. Chapman,

Thank you for contacting us regarding shipping 2 parcels from Austin to New York City. 

For 2 parcels, each weighing 500 grams, the total price for shipping would be $55.99. We estimate the delivery timeframe to be within 2 weeks from the ship date. 

Please let me know if this shipping price and timeline meets your needs. Also, could you please indicate if this shipment is for personal or business purposes? That information helps us select the correct insurance for your parcel delivery.

We appreciate your business. Let me know if you need any other information from me or are ready to move forward with booking this shipment. I'm happy to assist further.

Best regards,
Daniel Elliott
"""

query = f"""
<emails>
{emails}
</emails>

Use the extract_email tool.
"""

response = client.converse(
    modelId=MODEL_NAME,
    inferenceConfig={
        "temperature": 0,
        "maxTokens": 4000
    },
    toolConfig={"tools": tools},
    messages=[{"role": "user", "content": [{"text": query}]}]
)

json_entities = None
for content in response['output']['message']['content']:
    if "toolUse" in content and content['toolUse']['name'] == "extract_email":
        json_entities = content['toolUse']['input']
        break

if json_entities:
    print("Extracted Entities (JSON):")
    print(json.dumps(json_entities, indent=2))
else:
    print("No entities found in the response.")

Extracted Entities (JSON):
{
  "entities": [
    {
      "SenderName": "John Chapman",
      "SenderAddress": "2418 Spring Ln #5362 Austin, TX 78703 United States",
      "ReceiverName": "Anna Chapman",
      "ReceiverAddress": "240 E 27th St New York, NY 10016 United States",
      "NumberOfParcels": "2",
      "TotalWeightOfParcels": "1000 grams",
      "WeightPerParcels": "500 grams",
      "Price": 55.99,
      "PriceCurrency": "USD",
      "DeliveryTimeframe": "within 2 weeks"
    }
  ]
}
